In [1]:
import geopandas as gpd
import sqlite3
import pandas as pd

# Initial Steps

## Read shp files

In [2]:
gadm_df = gpd.read_file('gadm41_PHL_shp/gadm41_PHL_3.shp')

In [3]:
osm_df = gpd.read_file('philippines-latest-free.shp/gis_osm_pois_a_free_1.shp')

In [9]:
osm_pts_df = gpd.read_file('hotosm_phl_points_of_interest_points_shp/hotosm_phl_points_of_interest_points_shp.shp')

In [10]:
osm_pts_df

,name,name_en,amenity,man_made,shop,tourism,opening_ho,beds,rooms,addr_full,addr_house,addr_stree,addr_city,source,name_fil,osm_id,osm_type,geometry
0,None,None,ferry_terminal,None,None,None,None,None,None,None,None,None,None,None,None,8340552790,nodes,POINT (123.28010 12.22461)
1,Palawan Express,None,money_transfer,None,None,None,None,None,None,None,1,Juan Luna,None,None,None,10764757805,nodes,POINT (123.28357 12.22478)
2,None,None,doctors,None,None,None,None,None,None,None,None,Juan Luna,None,None,None,10765152820,nodes,POINT (123.28371 12.22485)
3,Mandaon Terminal,None,bus_station,None,None,None,Mo-Su 05:30-03:30,None,None,None,None,None,None,None,None,10764748805,nodes,POINT (123.28392 12.22497)
4,Iglesia Ni Cristo - Madalag Extension,None,place_of_worship,None,None,None,None,None,None,None,None,None,None,None,None,10297680713,nodes,POINT (122.03713 12.24144)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200579,JM Agrivet Supplies,None,None,None,animal_feed,None,None,None,None,None,None,None,None,None,None,11590899744,nodes,POINT (124.01200 10.72101)
200580,Villa Sandra,None,None,None,None,hostel,None,None,None,None,None,None,None,None,None,4161416389,nodes,POINT (124.11536 11.32997)
200581,Jumping cliff,None,None,None,None,viewpoint,None,None,None,None,None,None,None,None,None,6538984985,nodes,POINT (124.10627 11.34683)
200582,None,None,None,None,None,apartment,None,None,None,None,None,None,None,None,None,11590997769,nodes,POINT (120.83169 14.37424)


## Prep data before saving to avoid utf-8 errors when querying

In [12]:
for geo_df in [gadm_df, osm_df, osm_pts_df]:
    for col in geo_df.select_dtypes(include=['object']).columns:
        geo_df[col] = geo_df[col].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

## Save to map.sqlite

In [5]:
gadm_df.to_file("map.sqlite", driver="SQLite", layer="gadm_ph")

In [6]:
osm_df.to_file("map.sqlite", driver="SQLite", layer="osm_ph")

In [13]:
osm_pts_df.to_file("map.sqlite", driver="SQLite", layer="osm_ph_pts")

## Verify query if tables exist

In [14]:
conn = sqlite3.connect('map.sqlite')

In [15]:
sql = '''
SELECT name
FROM sqlite_master
WHERE type = 'table'
'''
pd.read_sql(sql, conn)

,name
0,geometry_columns
1,spatial_ref_sys
2,gadm_ph
3,sqlite_sequence
4,osm_ph
5,osm_ph_pts
